# Liga 1 Perú 2021: Base de datos

El siguiente script y base de datos responde a la falta de una base de datos totalizada que reporte información diversificada sobre los partidos de la primera división del futbol peruano y aportar con ideas para solucionar problemas que se reportan al elaborar 'scrapers'. Se espera que la base de datos resultante motive la exploración, análisis y desarrollo en temas relacionado con el sport analytics en el futbol peruano. Además, satisfacer la falta de una base de datos con variedad de datos y gratuita. La base de datos no puede ser usada con fines comerciales.

Si tiene alguna sugerencia o comentario, contactar mediante:
- @: gianfranco.soria@pucp.edu.pe
- GitHub: GianfrancoSoria
- Twitter: @SoriaGianfranco

## Librerías necesarias

En esta sección se importan todas las librerías que serán necesarias para el desarrollo del código más adelante.

In [19]:
#!pip install webdriver-manager
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd
import numpy as np
import threading

## Evaluación de la página

La página seleccionada para ser la fuente de datos es [Flashscore](https://www.flashscore.pe/futbol/peru/liga-1-2021/resultados/). La elección se debió a que la página ofrece los datos de forma gratuita, no tiene bloqueos o limitaciones y presenta una estructura idónea para la extracción de datos. 

Tras la evaluación de la página se encuentra que los resultados se muestran en dos partes. En la primera se muestran los resultados de diferentes partidos, pero solo algunos datos (fecha, número de goles y tarjetas rojas). En la segunda se muestra el detallado de algún partido seleccionado, los datos que se muestran son variados (goles, tarjetas amarillas, offside, etc.). Sin embargo, se reconocen algunos retos para ambas etapas. En la primera etapa se reconoce que no se muestran todos los partidos, solo algunos. Para acceder a todos se debe cargar los datos y para ello se requiere desplazar la página hasta el final y hacer clic en la opción "Mostrar más partidos". En la segunda etapa se observa que los vínculos (que se encuentran en la primera etapa) están ocultos, pero se identifica una estructura común en la dirección URL con "códigos identificadores" para cada partido.

Solucionar los retos de la primera parte se emplean los comandos "driver.execute_script" (rodar el scroll de la página) y "driver.find_element" (ubicar y hacer clic en "Mostrar más partidos"). En cuanto a los retos de la segunda parte resultan más difícil de solucionar, mas no imposibles. Para extraer los "códigos identificadores" emplearemos una estrategia similar a si usáramos la librería beautifulsoup. Delimitamos que estos códigos se encuentran entre 'div id="g_1_' y 'title="¡Haga click para detalles del partido!"'. Empleando esta restricción almacenamos los resultados en una lista con nombre "result_buscar".

In [20]:
#---------------------------------------------------------------------------------------------------------------------------------------------
# Apertura de página
#---------------------------------------------------------------------------------------------------------------------------------------------
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.flashscore.pe/futbol/peru/liga-1-2021/resultados/") 
driver.find_element(By.XPATH, "//*[@id='onetrust-accept-btn-handler']").click()
driver.maximize_window()
#---------------------------------------------------------------------------------------------------------------------------------------------
# Retos primera etapa
#---------------------------------------------------------------------------------------------------------------------------------------------
driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
driver.find_element(By.XPATH, "//*[@id='live-table']/div[1]/div/div/a").click()
time.sleep(2)
driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
driver.find_element(By.XPATH, "//*[@id='live-table']/div[1]/div/div/a").click()
time.sleep(2)
driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
#---------------------------------------------------------------------------------------------------------------------------------------------
# Retos segunda etapa
#---------------------------------------------------------------------------------------------------------------------------------------------
content = driver.find_element(By.XPATH, "//*[@id='live-table']/div[1]/div/div")
soup=content.get_attribute('innerHTML')
buscar1='<div id="g_1_'
result1 = [_.start() for _ in re.finditer(buscar1, soup)]
buscar2='title="¡Haga click para detalles del partido!"'
result2 = [_.start() for _ in re.finditer(buscar2, soup)]
result_buscar=[0]*len(result1)
for i in range(len(result1)):
    result_buscar[i]=str(soup[result1[i]+13:result2[i]-2])
driver.close()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache
C:\Users\PC-1\AppData\Local\Temp\ipykernel_9256\3492846082.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## Primeros resultados: Árbitros

Empleando los "códigos identificadores" extraídos anteriormente podemos visualizar las páginas personalizadas para cada partido. Estos resultados muestran 4 opciones (Partido, Cuotas, H2H y Cuadro). Los datos de interés se encuentran en la opción "Partido" y dentro de las sub opciones "Resumen" (nombre del árbitro) y "Estadísticas" (fecha, número de goles y tarjetas rojas). Como primer ejercicio de exploración nos centraremos en la extracción del nombre del árbitro. Se encontro que la estructura del html no es similar en todos, existe la excepción para los códigos del 22 al 31. Este hecho se soluciona con un condicional 'if' en este rango. Por otro lado, identificamos que el tiempo mínimo para para cada código identificador es de 9.5 segundos (contando solo los sleep) y ,por ello, el tiempo total mínimo sería 38 minutos. Este tiempo podría considerarse como extenso y representa un reto computacional importante. Para sobrepasar este reto usaremos la librería 'threading' que nos permite usar más procesadores o hilos y estará limitado por el número que posea nuestro procesador. Emplear más hilos o procesadores nos permite ejecutar múltiples tareas al mismo tiempo. Aplicado a este caso, implica que podremos scrapear varios códigos identificadores al mismo tiempo. El número de hilos usados en el presente proceso fue de 15 y el tiempo total fue 5.66 minutos. Este tiempo se vera afectado por el número de hilos y la conexión a internet.

In [21]:
arbitro=[0]*len(result_buscar)
def referee(min_i,max_i):
    for i in range(min_i,max_i):
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get('https://www.flashscore.pe/partido/'+f'{result_buscar[i]}'+'/#/resumen-del-partido/resumen-del-partido')
        time.sleep(6)
        driver.find_element(By.XPATH, "//*[@id='onetrust-accept-btn-handler']").click()
        driver.maximize_window()
        time.sleep(0.5)
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
        if i>21 and i<32:
            arbitro[i]=driver.find_element(By.XPATH,"//*[@id='detail']/div[10]/div[2]/div[1]/span[2]").text #Revisar aquí por actualizaciones
        else:
            arbitro[i]=driver.find_element(By.XPATH,"//*[@id='detail']/div[11]/div[2]/div[1]/span[2]").text #Revisar aquí por actualizaciones
        time.sleep(3)
        driver.close()
        print('Termino '+str(i))
        print(arbitro)
#---------------------------------------------------------------------------------------------------------------------------------------------
# Número total de threads usados
n_threads=15
#---------------------------------------------------------------------------------------------------------------------------------------------
p=len(result_buscar)//n_threads
inicios=list()
finales=list()
inicio=0
fin=p
for i in range(n_threads):
    inicios.append(inicio)
    finales.append(fin)
    inicio+=p
    fin+=p
time_inicio = time.time()
threads=list()
for j in range(len(inicios)):
    t=threading.Thread(target=referee, args=(inicios[j],finales[j],))
    threads.append(t)
    t.start()
for t in threads:
    t.join()
print(' ')
print('--------------------------------------------------------------------------------------------------------------------------------------')
print('El tiempo total fue de '+str(time.time()-time_inicio)+' segundos.')
print('--------------------------------------------------------------------------------------------------------------------------------------')

[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Get LATEST chrom

...

Termino 63
['Ortega K. (Per)', 'Haro D. (Per)', 'Ordonez E. (Per)', 'Espinoza M. (Per)', 'Alarcon J. (Per)', 'Huaman D. (Per)', 'Haro Sueldo D. M. (Per)', 'Espinoza M. (Per)', 'Estadio Mansiche (Trujillo)', 'Perez R. B. (Per)', 'Seminario L. (Per)', 'Palomino M. (Per)', 'Barbagelatta G. (Per)', 'Palomino M. (Per)', 'Espinoza M. (Per)', 0, 'Estadio Monumental (Lima)', 'Ordonez E. (Per)', 'Alarcon J. (Per)', 'Legario F. (Per)', 'Carrillo V. (Per)', 'Ortega K. (Per)', 'Estadio Mansiche (Trujillo)', 'Ordonez E. (Per)', 'Ortega K. (Per)', 'Legario F. (Per)', 'Menendez A. (Per)', 'Casanova V. (Per)', 'Estadio Mansiche (Trujillo)', 'Estadio Héroes de San Ramón (Cajamarca)', 'Palomino M. (Per)', 'Cartagena J. (Per)', 'Carrillo V. (Per)', 'Perez R. B. (Per)', 'Huaman D. (Per)', 'Palomino M. (Per)', 'Menendez A. (Per)', 'Legario F. (Per)', 'Espinoza J. (Per)', 'Seminario L. (Per)', 'Lopez P. (Per)', 'Villegas H. (Per)', 'Lopez Ramos P. (Esp)', 'Estadio Campeones del 36 (Sullana)', 'Garay L. (Per

[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


Termino 175
['Ortega K. (Per)', 'Haro D. (Per)', 'Ordonez E. (Per)', 'Espinoza M. (Per)', 'Alarcon J. (Per)', 'Huaman D. (Per)', 'Haro Sueldo D. M. (Per)', 'Espinoza M. (Per)', 'Estadio Mansiche (Trujillo)', 'Perez R. B. (Per)', 'Seminario L. (Per)', 'Palomino M. (Per)', 'Barbagelatta G. (Per)', 'Palomino M. (Per)', 'Espinoza M. (Per)', 0, 'Estadio Monumental (Lima)', 'Ordonez E. (Per)', 'Alarcon J. (Per)', 'Legario F. (Per)', 'Carrillo V. (Per)', 'Ortega K. (Per)', 'Estadio Mansiche (Trujillo)', 'Ordonez E. (Per)', 'Ortega K. (Per)', 'Legario F. (Per)', 'Menendez A. (Per)', 'Casanova V. (Per)', 'Estadio Mansiche (Trujillo)', 'Estadio Héroes de San Ramón (Cajamarca)', 'Palomino M. (Per)', 'Cartagena J. (Per)', 'Carrillo V. (Per)', 'Perez R. B. (Per)', 'Huaman D. (Per)', 'Palomino M. (Per)', 'Menendez A. (Per)', 'Legario F. (Per)', 'Espinoza J. (Per)', 'Seminario L. (Per)', 'Lopez P. (Per)', 'Villegas H. (Per)', 'Lopez Ramos P. (Esp)', 'Estadio Campeones del 36 (Sullana)', 'Garay L. (Pe

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


Termino 79
['Ortega K. (Per)', 'Haro D. (Per)', 'Ordonez E. (Per)', 'Espinoza M. (Per)', 'Alarcon J. (Per)', 'Huaman D. (Per)', 'Haro Sueldo D. M. (Per)', 'Espinoza M. (Per)', 'Estadio Mansiche (Trujillo)', 'Perez R. B. (Per)', 'Seminario L. (Per)', 'Palomino M. (Per)', 'Barbagelatta G. (Per)', 'Palomino M. (Per)', 'Espinoza M. (Per)', 'Perez R. B. (Per)', 'Estadio Monumental (Lima)', 'Ordonez E. (Per)', 'Alarcon J. (Per)', 'Legario F. (Per)', 'Carrillo V. (Per)', 'Ortega K. (Per)', 'Estadio Mansiche (Trujillo)', 'Ordonez E. (Per)', 'Ortega K. (Per)', 'Legario F. (Per)', 'Menendez A. (Per)', 'Casanova V. (Per)', 'Estadio Mansiche (Trujillo)', 'Estadio Héroes de San Ramón (Cajamarca)', 'Palomino M. (Per)', 'Cartagena J. (Per)', 'Carrillo V. (Per)', 'Perez R. B. (Per)', 'Huaman D. (Per)', 'Palomino M. (Per)', 'Menendez A. (Per)', 'Legario F. (Per)', 'Espinoza J. (Per)', 'Seminario L. (Per)', 'Lopez P. (Per)', 'Villegas H. (Per)', 'Lopez Ramos P. (Esp)', 'Estadio Campeones del 36 (Sullana

Explorando los resultados encontramos que para algunos casos se extrae el nombre del estadio y no del árbitro. La razón de este hecho es que para dichos partidos no se registró el nombre del árbitro, por lo que no hay acción correctiva alguna. Lo que sí se puede corregir es cambiar dichos nombres por valores faltantes, representados por "-". Identificamos estos nombres en la lista "agregar_NaN".

In [22]:
agregar_NaN=['Estadio Miguel Grau (Callao)','Estadio Mansiche (Trujillo)','Estadio Campeones del 36 (Sullana)',
            'Estadio Héroes de San Ramón (Cajamarca)','Estadio Inca Garcilaso de la Vega (Cusco)','Estadio San Martin de Porres (Lima)',
            'Estadio Huancayo (Huancayo)','Estadio Guillermo Briceno Rosamedina (Juliaca)','Estadio Alejandro Villanueva (Lima)',
            'Estadio Monumental (Lima)']
for j in agregar_NaN:
    for i in range(len(arbitro)):
        if arbitro[i]==j:
            arbitro[i]='-'
print(arbitro)

['Ortega K. (Per)', 'Haro D. (Per)', 'Ordonez E. (Per)', 'Espinoza M. (Per)', 'Alarcon J. (Per)', 'Huaman D. (Per)', 'Haro Sueldo D. M. (Per)', 'Espinoza M. (Per)', '-', 'Perez R. B. (Per)', 'Seminario L. (Per)', 'Palomino M. (Per)', 'Barbagelatta G. (Per)', 'Palomino M. (Per)', 'Espinoza M. (Per)', 'Perez R. B. (Per)', '-', 'Ordonez E. (Per)', 'Alarcon J. (Per)', 'Legario F. (Per)', 'Carrillo V. (Per)', 'Ortega K. (Per)', '-', 'Ordonez E. (Per)', 'Ortega K. (Per)', 'Legario F. (Per)', 'Menendez A. (Per)', 'Casanova V. (Per)', '-', '-', 'Palomino M. (Per)', 'Cartagena J. (Per)', 'Carrillo V. (Per)', 'Perez R. B. (Per)', 'Huaman D. (Per)', 'Palomino M. (Per)', 'Menendez A. (Per)', 'Legario F. (Per)', 'Espinoza J. (Per)', 'Seminario L. (Per)', 'Lopez P. (Per)', 'Villegas H. (Per)', 'Lopez Ramos P. (Esp)', '-', 'Garay L. (Per)', 'Huaman D. (Per)', 'Legario F. (Per)', 'Menendez A. (Per)', 'Perez R. B. (Per)', 'Ordonez E. (Per)', 'Garay Evia L. (Per)', 'Perez R. B. (Per)', 'Menendez A. (Per

Explorando los resultados se determinó que el número de NaN que se han colocado es 20 y representa el 8.33%. Asimismo, en la siguiente tabla se explora el número de partidos por arbitro que se registra.

In [23]:
values, counts = np.unique(arbitro, return_counts=True)
data = {'Arbitro': list(values), 'Partidos': list(counts)}
tabla_arbitro=pd.DataFrame(data).sort_values('Partidos',ascending=False).reset_index(drop=True)
print('La cantidad de NaN es '+str(tabla_arbitro.Partidos[0])+' ('+ str(round(tabla_arbitro.Partidos[0]/tabla_arbitro.Partidos.sum()*100,2))+' %).')
tabla_arbitro.drop(tabla_arbitro.index[[0]])

La cantidad de NaN es 20 (8.33 %).


,Arbitro,Partidos
1,Espinoza M. (Per),17
2,Legario F. (Per),16
3,Perez R. B. (Per),16
4,Ordonez E. (Per),16
5,Palomino M. (Per),15
6,Menendez A. (Per),15
7,Alarcon J. (Per),15
8,Cartagena J. (Per),12
9,Ortega K. (Per),12
10,Quiroz J. (Per),10


## Extracción de multiples datos y creación de base

En esta sección nos centraremos en extraer datos relacionados con el partido y equipos. Los datos de interés se encuentran en una sub pestaña de nombre "Estadísticas". Explorando las opciones para acceder a esta pestaña se optó por proceder mediante la dirección "url". Para ello necesitamos añadir a la dirección anteriormente usada el siguiente texto: '/#/resumen-del-partido/estadisticas-del-partido/0'. Tras explorar las diversas opciones para extraer los datos y las similitudes con las secciones anteriores se diseñó una estrategia muy similar. Se crearon tres bases previas para almacenar los datos: dt_local, almacenara los datos del equipo local; dt_visita, almacenara los datos del equipo de visita y key_list, almacena el nombre de las variables de interés. El primer reto que se encontró está relacionado con las variables de interés puesto que dichas variables no se reportan o mantienen el mismo orden. Para ello se optó por extraer toda la sección y extraer los datos de manera similar que para los árbitros. Para fijar los limites transformamos la lista de almacenada como texto a lista, separado por '\n', y la lista 'key_list'. En conjunto nos permiten separar las variables de interés y fijar un orden determinado. El siguiente reto estaba relacionado con la estructura de la página, como ya se había reportado antes, no es similar para todos los casos. Se encontró que existe la excepción para los códigos del 22 al 31. Este hecho se soluciona con un condicional 'if' en este rango. Asimismo, se mantuvo la estrategia del uso de "threads" por razones antes expuestas en la sección anterior. El número de hilos usados en el presente proceso fue de 15 y el tiempo total fue 6.60 minutos.

In [37]:
dt_local=pd.DataFrame(columns=['Fecha','Local','GL','Posesión de balón','Remates','Remates a puerta','Remates fuera','Remates rechazados','Córneres',
         'Fueras de juego','Paradas','Faltas','Tarjetas rojas','Tarjetas amarillas','Pases totales','Tackles',
          'Ataques','Ataques peligrosos'], index=range(len(result_buscar)))
dt_visita=pd.DataFrame(columns=['Visita','GV','Posesión de balón','Remates','Remates a puerta','Remates fuera','Remates rechazados','Córneres',
         'Fueras de juego','Paradas','Faltas','Tarjetas rojas','Tarjetas amarillas','Pases totales','Tackles',
          'Ataques','Ataques peligrosos'], index=range(len(result_buscar)))
key_list=['Posesión de balón','Remates','Remates a puerta','Remates fuera','Remates rechazados','Córneres',
         'Fueras de juego','Paradas','Faltas','Tarjetas rojas','Tarjetas amarillas','Pases totales','Tackles',
          'Ataques','Ataques peligrosos']

In [38]:
def team(min_i,max_i):
    for i in range(min_i,max_i):
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get('https://www.flashscore.pe/partido/'+f'{result_buscar[i]}'+'/#/resumen-del-partido/estadisticas-del-partido/0')
        time.sleep(8)
        driver.find_element(By.XPATH, "//*[@id='onetrust-accept-btn-handler']").click()
        driver.maximize_window()
        time.sleep(0.5)
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
        if i>=22 and i<=31:
            soup=driver.find_element(By.XPATH,'//*[@id="detail"]/div[8]').text.split('\n') #Revisar aquí por actualizaciones
        else:
            soup=driver.find_element(By.XPATH,'//*[@id="detail"]/div[9]').text.split('\n') #Revisar aquí por actualizaciones
        dt_local.loc[i,'Fecha']=driver.find_element(By.XPATH,"//*[@id='detail']/div[4]/div[1]/div").text #Revisar aquí por actualizaciones
        dt_local.loc[i,'Local']=driver.find_element(By.XPATH,"//*[@id='detail']/div[4]/div[2]/div[3]/div[2]/a").text #Revisar aquí por actualizaciones
        dt_visita.loc[i,'Visita']=driver.find_element(By.XPATH,"//*[@id='detail']/div[4]/div[4]/div[3]/div[1]/a").text #Revisar aquí por actualizaciones
        dt_local.loc[i,'GL']=driver.find_element(By.XPATH,"//*[@id='detail']/div[4]/div[3]/div/div[1]/span[1]").text #Revisar aquí por actualizaciones
        dt_visita.loc[i,'GV']=driver.find_element(By.XPATH,"//*[@id='detail']/div[4]/div[3]/div/div[1]/span[3]").text #Revisar aquí por actualizaciones
        key_list_ind=list()
        key_list_name=list()
        for j in range(len(key_list)):
            if key_list[j] in soup:
                key_list_ind.append(soup.index(key_list[j]))
                key_list_name.append(key_list[j])
        for k in range(len(key_list_ind)):
            dt_local.loc[i,key_list_name[k]]=soup[key_list_ind[k]-1]
            dt_visita.loc[i,key_list_name[k]]=soup[key_list_ind[k]+1]
        time.sleep(5)
        driver.close()
#---------------------------------------------------------------------------------------------------------------------------------------------
# Número total de threads usados
n_threads=15
#---------------------------------------------------------------------------------------------------------------------------------------------p=len(result_buscar)//n_threads
inicios=list()
finales=list()
inicio=0
fin=p
for i in range(n_threads):
    inicios.append(inicio)
    finales.append(fin)
    inicio+=p
    fin+=p
time_inicio = time.time()
threads=list()
for j in range(len(inicios)):
    t=threading.Thread(target=team, args=(inicios[j],finales[j],))
    threads.append(t)
    t.start()
for t in threads:
    t.join()
print(' ')
print('--------------------------------------------------------------------------------------------------------------------------------------')
print('El tiempo total fue de '+str(time.time()-time_inicio)+' segundos.')
print('--------------------------------------------------------------------------------------------------------------------------------------')

[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======


[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Current google-chrome version is 

...

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\PC-1\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


 
--------------------------------------------------------------------------------------------------------------------------------------
El tiempo total fue de 396.13586497306824 segundos.
--------------------------------------------------------------------------------------------------------------------------------------


Como se había mencionado, se crearon dos data frame para almacenar los resultados ('dt_visita' y 'dt_local'). Ambos data frame se unirán en uno solo de nombre 'dt', con la finalidad de tener una base totalizada.

In [39]:
dt_visita.columns=['Visita','GV','PV','RemV','RpV','RfV','RbV','CV',
                    'OffV','ArqV','FV','RV','AV','TpV','BkV','AtaqV',
                   'DAtaqV']
dt_local.columns=['Fecha','Local','GL','PL','RemL','RpL','RfL','RbL','CL',
                   'OffL','ArqL','FL','RL','AL','TpL','BkL','AtaqL',
                   'DAtaqL']
#---------------------------------------------------------------------------------------------------------------------------------------------
# Colapsar bases de datos (local y visitante) en una sola
dt=pd.merge(dt_local, dt_visita, left_index=True, right_index=True).reindex(columns=['Fecha','Local','Visita','GL','GV','PL','PV','RemL','RemV',
                                                                                     'RpL','RpV','RfL','RfV','RbL','RbV','CL','CV','OffL','OffV',
                                                                                     'ArqL','ArqV','FL','FV','RL','RV','AL','AV','TpL','TpV',
                                                                                     'BkL','BkV','AtaqL','AtaqV','DAtaqL','DAtaqV'])
#---------------------------------------------------------------------------------------------------------------------------------------------

Explorando el data frame 'dt' se encontró que las variables relacionadas con las tarjetas del partido presentaban una múltiple cantidad de 'NaN'. Esto se explica debido a que no en todos los partidos se reportan tarjetas amarillas o rojas (especialmente). Estas 'NaN' fueron reemplazadas por el valor de 0, puesto que mantiene la premisa anterior. Por otro lado, las otras variables también reportaron 'NaN', sin embargo, no se puede reemplazar por el valor de 0. Puesto que no necesariamente siguen la misma lógica que las tarjetas del partido. Por lo que las demás 'NaN' fueron reemplazadas por '-' para no alterar su significado. Por último, se agregó la información de árbitros a la base totalizada.

In [40]:
# Últimos arreglos
dt['AL']=dt['AL'].fillna(0)
dt['AV']=dt['AV'].fillna(0)
dt['RL']=dt['RL'].fillna(0)
dt['RV']=dt['RV'].fillna(0)
dt=dt.fillna('-')
dt.insert(1,'Arbitro',arbitro)

## Exportar base de datos

In [41]:
dt.to_excel('Liga1 (Perú) 2021.xlsx', index=False, sheet_name='2021')
dt.to_csv('Liga1 (Perú) 2021.csv', index=False, sep=';')

## Nota final

Durante el desarrollo del presente script se realizaron hasta dos actualizaciones en el código relacionados con los XPATH. Por lo que es importante realizar un seguimiento al código para que no pierda validez. Asimismo, se ha encontrado que la estructura html no se mantiene para otros años. Por lo que si se quisiera obtener información de otro año se recomienda tomar como base el presente código y adaptar la nueva estructura html (XPATH por lo general).